<a href="https://colab.research.google.com/github/komal83/phdthesis/blob/main/LDA_topic_Modeling_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Stuff
!pip install spacy_udpipe
!pip install pyLDAvis
import nltk; nltk.download('stopwords')
import os
import glob
import matplotlib
import matplotlib.pyplot as plt
from google.colab import drive
import re
import bz2
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
import spacy_udpipe
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
#@title Drive Mount and data path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
extension = 'bz2'
os.chdir("<>")

In [ ]:
#@title Data Input corretto
def getData(start,end):
    df_list = list()
    for d in pd.date_range(start,end):
        filename = d.strftime('40wita_%Y-%m-%d.csv.bz2')
        data = pd.read_csv(filename)
        yield(data)
        #print(filename)
        #df_list.append(pd.read_csv(filename))
        #df = pd.concat(df_list)
        #df.head()
        #print(df.size)
        #df.head()
        #yield(df) 

In [ ]:
data=[]
id=[]
text=[]
input_data =[]
day=[]
for foo in getData('2020-02-01', '2020-04-30'):
        #input_data= foo
        #x = list(foo)
        id.append(foo['id'])
        text.append(foo['text'])
        text=foo.text.values.tolist()
        #text.append(foo.text.values.tolist())
        #text=foo.text.values.tolist()
        #print(id[0:3])
        day.append(foo['day'])
        #print(day[0:3])
        #print(foo.day[0:3])
        #print(x.day)
data_input=list(zip(id,text,day))
#print(day)   
#print(data_input[0:3])     
#print(input_data.day[0:4])
#pprint(text)

In [ ]:
#pprint(text[0:3])

In [ ]:
#@title NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('italian')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
#@title url removal
# Convert to list
data = text
# Remove url
data = [re.sub(r"http\S+", "", sent) for sent in data]

#pprint(data[:1])

In [ ]:
#@title def funzione tokenizzazione e stop words removal


    
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]



In [ ]:
#@title Pre tokenizzazione con gensim
data_words = list(sent_to_words(data))
#pprint(data_words[:1])

In [ ]:
#@title Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
#print(data_words_nostops[:1])

In [ ]:
#@title Def lemmatization Function
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp=spacy_udpipe.load_from_path(lang="it",
                                  path="<italian-postwita-ud-2.5-191206.udpipe>",
                                  meta={"description": "Custom 'it' model"}, disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
#@title  Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
#@title LDA Function

def lda(num_topics,chunksize,passes,eval_every):
    #result=pd.DataFrame()
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           eval_every=eval_every,
                                           alpha='auto',
                                           per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    perplexity = lda_model.log_perplexity(corpus)
    result= {   
        'perplexity': perplexity, 
        'coherence_score':coherence_lda, 
        'num_topics':num_topics,
        'chunksize':chunksize,
        'passes':passes
    }
    print('\nCoherence Score: ', coherence_lda)
    print('\nPerplexity: ', perplexity)
    print(result)
    #pprint(lda_model.print_topics())
    return lda_model, result

In [ ]:
#@title LDA run on one single set of parameter
'''foo=lda(50,50,10,1)
#pprint(foo)
result=foo.DataFrame()'''

In [ ]:
#@title LDA cycle over different parameters: num_topics,chunksize,passes,eval_every
result_df=pd.DataFrame()    
for i in range(50):
    #lda(num_topics,chunksize,passes,eval_every)
    lda_model, lda_result=lda( i+1, 50,10,10)
    result_df=result_df.append(lda_result, ignore_index=True)    

In [ ]:
#print(result_df)
#print(result_df['coherence_score'])

In [ ]:
#@title Plot coherence score
plt.plot(result_df['num_topics'],result_df['coherence_score'])
plt.xlabel('Num topics') 
plt.ylabel('coherence score') 
plt.title("Coherence 50 Topics all dataset")
plt.grid()
plt.savefig("<coherence_figure>")
plt.close()

In [ ]:
#@title Plot perplexity score
plt.plot(result_df['num_topics'],result_df['perplexity'])
plt.xlabel('Num topics') 
plt.ylabel('Perplexity') 
plt.title("Perplexity 50 Topics all dataset")
plt.grid()
plt.savefig("<perplexity figure>")
plt.close()

In [ ]:
with open('/content/drive/My Drive/COVID-19/results_50.csv','a' ) as fd:
    result_df.to_csv(fd, index= True)

In [ ]:
#@title  save the model
lda_model.save('<lda model path>')

In [ ]:
#@title Print the Keyword in the N topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#@title  Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14     0.084435  0.249370       1        1  7.971636
2      0.373116 -0.136048       2        1  7.480345
15     0.061786  0.264377       3        1  6.655283
4      0.010814  0.010917       4        1  6.058088
5     -0.011134 -0.019097       5        1  5.891833
1     -0.030929 -0.024621       6        1  5.096197
3     -0.022471 -0.024312       7        1  5.071797
7     -0.026690 -0.022839       8        1  5.038411
9     -0.037147 -0.020208       9        1  4.784648
17    -0.025353 -0.014591      10        1  4.771996
13    -0.041167 -0.026025      11        1  4.621942
6     -0.033372 -0.023293      12        1  4.614625
10    -0.039556 -0.025629      13        1  4.566985
12    -0.024862 -0.026322      14        1  4.563425
16    -0.025353 -0.027383      15        1  4.453968
8     -0.034174 -0.017892      16        1  4.248828
0     -0.041713 -0.030613      17        1  3.778346
18    -0.044447 -0.025240      18        1  3.678109
11    -0.043949 -0.029682      19        1  3.482586
19    -0.047834 -0.030868      20        1  3.170953, topic_info=              Term         Freq        Total Category  logprob  loglift
22            fare  2421.000000  2421.000000  Default  30.0000  30.0000
285     quarantena  2035.000000  2035.000000  Default  29.0000  29.0000
10           nuovo  1451.000000  1451.000000  Default  28.0000  28.0000
189            piu  1639.000000  1639.000000  Default  27.0000  27.0000
134    coronavirus  1333.000000  1333.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
4112  interessante    42.280369    43.134792  Topic20  -5.1172   3.4311
4382   particolare    41.767258    42.621681  Topic20  -5.1294   3.4309
2882      telefono    41.096691    41.951115  Topic20  -5.1456   3.4306
810           roba    39.250450    40.104874  Topic20  -5.1915   3.4296
3645    democrazia    37.763004    38.617428  Topic20  -5.2302   3.4288

[642 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
2842       8  0.976082        accesso
1104      19  0.992476        accordo
2899       7  0.988147          acqua
2315       2  0.990571     affrontare
801        1  0.998782  aggiornamento
...      ...       ...            ...
2857      12  0.986339     volontario
1110      17  0.996825          volta
4785      12  0.972306          volto
77        20  0.985954        youtuba
14537      2  0.985988         zalone

[672 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 3, 16, 5, 6, 2, 4, 8, 10, 18, 14, 7, 11, 13, 17, 9, 1, 19, 12, 20])

In [ ]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])

pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("<file path>")


In [ ]:
#@title Load saved model and print topic from there
## Save model to disk.
#lda = LdaModel.load(<model path>)
#pprint(lda_model.print_topics())